In [1]:
import cv2
import os
import sys

import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Data Path
root1 = 'data/'
root2 = 'data_pre_processing512/'
TRAIN_PATH = 'stage1_train/'
TEST_PATH = 'stage1_test/'


In [3]:
# Get train and test IDs
train_ids = next(os.walk(root1+TRAIN_PATH))[1]
test_ids = next(os.walk(root1+TEST_PATH))[1]

In [4]:
print (len(train_ids))
print (len(test_ids))


660
65


In [5]:
import math
pad_size = 512
resize_size = 512
kernel = np.ones((4,4),np.uint8)

for n, id_ in enumerate(train_ids):
        path = root1 + TRAIN_PATH + id_
        img = cv2.imread(path + '/images/' + id_ + '.png')
        mean_value = np.mean(img.reshape(-1,3),axis=0).astype('uint8').tolist()
        
        value = tuple(mean_value)
        print (value,type(value))
        mask = np.zeros(img.shape[:2],dtype=np.uint8)
        
        edge = np.zeros(img.shape[:2],dtype=np.uint8)
        edge_ = np.zeros(img.shape[:2],dtype=np.uint8)
        overlap = np.zeros(img.shape[:2],dtype=np.uint8)
        overlap_ = np.zeros(img.shape[:2],dtype=np.float)
        gt = np.zeros(img.shape[:2],dtype=np.uint8)
        
        print (n,img.shape,img.dtype)
        img_short = min(img.shape[:2])
        img_long = max(img.shape[:2])
        
        if img_long < pad_size-8 :    
            pady1 = int((pad_size - img.shape[0]-8)*0.5)
            padx1 = int((pad_size - img.shape[1]-8)*0.5)
            pady2 =int(math.ceil((pad_size - img.shape[0]-8)*0.5))
            padx2 =int(math.ceil((pad_size - img.shape[1]-8)*0.5))
            img = cv2.copyMakeBorder(img,4,4,4,4,cv2.BORDER_CONSTANT,value=tuple(mean_value))
            img = cv2.copyMakeBorder(img,pady1,pady2,padx1,padx2,cv2.BORDER_REFLECT)
            
        if img_short >= resize_size :
            img = cv2.resize(img, (resize_size,resize_size), interpolation = cv2.INTER_LINEAR)
        
        save_dir = root2 + TRAIN_PATH + id_ + '/images/'
        save_path = save_dir + id_ + '.png'
        if not os.path.exists(save_dir) : os.makedirs(save_dir)
        cv2.imwrite(save_path,img)
               
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = cv2.imread(path + '/masks/' + mask_file,cv2.IMREAD_GRAYSCALE)
            area = np.array(mask_, dtype=np.float)/255
            small = np.sum(area)
            if small<=40 :
                print (small, 'delet')
                continue
            
            ret,thresh = cv2.threshold(mask_,127,255,0)
            dilation = cv2.dilate(thresh,kernel,iterations = 1)
            
            overlap_ +=  (dilation/255.0)
            
            image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)                       
            edge_ = cv2.drawContours(edge_, contours, -1,255, 3)
                        
            mask = np.maximum(mask, mask_)
            edge = np.maximum(edge, edge_)
        
        overlap = ((overlap_>1.0)*255).astype(np.uint8)
        gt = mask - overlap
        
        print ('mask',mask.shape,mask.dtype)
        mask_short = min(mask.shape[:2])
        mask_long = max(mask.shape[:2])
        
        if mask_long < pad_size-8 :    
            pady1 = int((pad_size - gt.shape[0]-8)*0.5)
            padx1 = int((pad_size - gt.shape[1]-8)*0.5)
            pady2 =int(math.ceil((pad_size - gt.shape[0]-8)*0.5))
            padx2 =int(math.ceil((pad_size - gt.shape[1]-8)*0.5))
            
            mask = cv2.copyMakeBorder(mask,4,4,4,4,cv2.BORDER_CONSTANT,value=0)
            mask = cv2.copyMakeBorder(mask,pady1,pady2,padx1,padx2,cv2.BORDER_REFLECT)
            edge = cv2.copyMakeBorder(edge,4,4,4,4,cv2.BORDER_CONSTANT,value=0)
            edge = cv2.copyMakeBorder(edge,pady1,pady2,padx1,padx2,cv2.BORDER_REFLECT)
            overlap = cv2.copyMakeBorder(overlap,4,4,4,4,cv2.BORDER_CONSTANT,value=0)
            overlap = cv2.copyMakeBorder(overlap,pady1,pady2,padx1,padx2,cv2.BORDER_REFLECT)
            gt = cv2.copyMakeBorder(gt,4,4,4,4,cv2.BORDER_CONSTANT,value=0)
            gt = cv2.copyMakeBorder(gt ,pady1,pady2,padx1,padx2,cv2.BORDER_REFLECT)
        
        if mask_short >= resize_size :
            mask = cv2.resize(mask,(resize_size,resize_size), interpolation = cv2.INTER_NEAREST)
            edge = cv2.resize(edge,(resize_size,resize_size), interpolation = cv2.INTER_NEAREST)
            overlap = cv2.resize(overlap,(resize_size,resize_size), interpolation = cv2.INTER_NEAREST)
            gt = cv2.resize(gt,(resize_size,resize_size), interpolation = cv2.INTER_NEAREST)
        
        if gt.shape[:2]!=(512,512):  print ('gt_size:',gt.shape,gt.dtype)
        
        save_dir = root2 + TRAIN_PATH + id_ + '/masks/'
        save_path = save_dir + id_ + '.png'
        if not os.path.exists(save_dir) : os.makedirs(save_dir)
        cv2.imwrite(save_path,mask)
        
        save_dir = root2 + TRAIN_PATH + id_ + '/edges/'
        save_path = save_dir + id_ + '.png'
        if not os.path.exists(save_dir) : os.makedirs(save_dir)
        cv2.imwrite(save_path,edge)
        
        save_dir = root2 + TRAIN_PATH + id_ + '/overlaps/'
        save_path = save_dir + id_ + '.png'
        if not os.path.exists(save_dir) : os.makedirs(save_dir)
        cv2.imwrite(save_path,overlap)
         
        save_dir = root2 + TRAIN_PATH + id_ + '/gts/'
        save_path = save_dir + id_ + '.png'
        if not os.path.exists(save_dir) : os.makedirs(save_dir)
        cv2.imwrite(save_path,gt)
              

(197, 134, 161) <class 'tuple'>
0 (256, 320, 3) uint8
mask (256, 320) uint8
(10, 10, 10) <class 'tuple'>
1 (256, 256, 3) uint8
37.0 delet
21.0 delet
mask (256, 256) uint8
(7, 7, 7) <class 'tuple'>
2 (512, 640, 3) uint8
mask (512, 640) uint8
(24, 24, 24) <class 'tuple'>
3 (520, 696, 3) uint8
37.0 delet
mask (520, 696) uint8
(202, 174, 185) <class 'tuple'>
4 (256, 320, 3) uint8
33.0 delet
21.0 delet
40.0 delet
mask (256, 320) uint8
(20, 20, 20) <class 'tuple'>
5 (520, 696, 3) uint8
37.0 delet
28.0 delet
25.0 delet
mask (520, 696) uint8
(9, 9, 9) <class 'tuple'>
6 (256, 256, 3) uint8
mask (256, 256) uint8
(6, 6, 6) <class 'tuple'>
7 (256, 256, 3) uint8
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
31.0 delet
21.0 delet
mask (256, 256) uint8
(44, 44, 44) <class 'tuple'>
8 (520, 696, 3) uint8
26.0 delet
40.0 delet
mask (520, 696) uint8
(29, 29, 29) <class 'tuple'>
9 (520, 696, 3) uint8
29.0 delet
mask (520, 696) uint8
(12, 12, 12) <class 'tuple'>
10 (256, 256, 3) uint8
m

38.0 delet
mask (360, 360) uint8
(3, 3, 3) <class 'tuple'>
88 (512, 640, 3) uint8
mask (512, 640) uint8
(9, 9, 9) <class 'tuple'>
89 (256, 256, 3) uint8
23.0 delet
26.0 delet
25.0 delet
mask (256, 256) uint8
(13, 13, 13) <class 'tuple'>
90 (256, 256, 3) uint8
21.0 delet
21.0 delet
21.0 delet
21.0 delet
26.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
38.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
22.0 delet
21.0 delet
21.0 delet
21.0 delet
40.0 delet
mask (256, 256) uint8
(203, 133, 160) <class 'tuple'>
91 (256, 320, 3) uint8
26.0 delet
32.0 delet
29.0 delet
25.0 delet
mask (256, 320) uint8
(17, 17, 17) <class 'tuple'>
92 (520, 696, 3) uint8
mask (520, 696) uint8
(19, 19, 19) <class 'tuple'>
93 (256, 256, 3) uint8
25.0 delet
mask (256, 256) uint8
(10, 10, 10) <class 'tuple'>
94 (360, 360, 3) uint8
mask (360, 360) uint8
(36, 36, 36) <class 'tuple'>
95 (520, 696, 3) uint8
36.0 delet
26.0 delet
mask (520, 696) uint8
(6, 

mask (360, 360) uint8
(13, 13, 13) <class 'tuple'>
169 (256, 256, 3) uint8
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
23.0 delet
21.0 delet
21.0 delet
21.0 delet
mask (256, 256) uint8
(18, 18, 18) <class 'tuple'>
170 (256, 256, 3) uint8
31.0 delet
21.0 delet
23.0 delet
39.0 delet
mask (256, 256) uint8
(208, 208, 208) <class 'tuple'>
171 (1024, 1024, 3) uint8
36.0 delet
39.0 delet
36.0 delet
21.0 delet
37.0 delet
24.0 delet
30.0 delet
34.0 delet
38.0 delet
36.0 delet
38.0 delet
mask (1024, 1024) uint8
(10, 10, 10) <class 'tuple'>
172 (360, 360, 3) uint8
36.0 delet
40.0 delet
30.0 delet
24.0 delet
mask (360, 360) uint8
(8, 8, 8) <class 'tuple'>
173 (256, 256, 3) uint8
mask (256, 256) uint8
(25, 25, 25) <class 'tuple'>
174 (256, 256, 3) uint8
21.0 delet
26.0 delet
30.0 delet
21.0 delet
23.0 delet
37.0 delet
21.0 delet
31.0 delet
30.0 delet
30.0 delet
21.0 delet
23.0 delet
30.0 delet
35.0 delet
21.0 delet
mask (256, 256) uint8
(10, 10, 10) <class 'tuple'>
175 (360, 360, 3) uint

mask (256, 320) uint8
(8, 8, 8) <class 'tuple'>
248 (256, 256, 3) uint8
mask (256, 256) uint8
(15, 15, 15) <class 'tuple'>
249 (256, 256, 3) uint8
21.0 delet
mask (256, 256) uint8
(5, 5, 5) <class 'tuple'>
250 (256, 256, 3) uint8
35.0 delet
24.0 delet
mask (256, 256) uint8
(14, 14, 14) <class 'tuple'>
251 (256, 256, 3) uint8
mask (256, 256) uint8
(10, 10, 10) <class 'tuple'>
252 (520, 696, 3) uint8
28.0 delet
mask (520, 696) uint8
(10, 10, 10) <class 'tuple'>
253 (256, 256, 3) uint8
21.0 delet
31.0 delet
mask (256, 256) uint8
(10, 10, 10) <class 'tuple'>
254 (360, 360, 3) uint8
32.0 delet
40.0 delet
29.0 delet
22.0 delet
39.0 delet
mask (360, 360) uint8
(5, 5, 5) <class 'tuple'>
255 (256, 256, 3) uint8
mask (256, 256) uint8
(10, 10, 10) <class 'tuple'>
256 (256, 256, 3) uint8
28.0 delet
33.0 delet
mask (256, 256) uint8
(16, 16, 16) <class 'tuple'>
257 (256, 256, 3) uint8
22.0 delet
mask (256, 256) uint8
(5, 5, 5) <class 'tuple'>
258 (256, 256, 3) uint8
mask (256, 256) uint8
(205, 169, 

mask (256, 256) uint8
(197, 127, 157) <class 'tuple'>
332 (256, 320, 3) uint8
mask (256, 320) uint8
(38, 38, 38) <class 'tuple'>
333 (260, 347, 3) uint8
36.0 delet
24.0 delet
38.0 delet
30.0 delet
24.0 delet
35.0 delet
23.0 delet
21.0 delet
32.0 delet
mask (260, 347) uint8
(29, 29, 29) <class 'tuple'>
334 (520, 696, 3) uint8
25.0 delet
26.0 delet
29.0 delet
mask (520, 696) uint8
(195, 116, 148) <class 'tuple'>
335 (256, 320, 3) uint8
24.0 delet
25.0 delet
mask (256, 320) uint8
(193, 95, 136) <class 'tuple'>
336 (256, 320, 3) uint8
mask (256, 320) uint8
(8, 8, 8) <class 'tuple'>
337 (520, 696, 3) uint8
32.0 delet
37.0 delet
27.0 delet
22.0 delet
35.0 delet
mask (520, 696) uint8
(216, 216, 216) <class 'tuple'>
338 (1024, 1024, 3) uint8
37.0 delet
33.0 delet
26.0 delet
38.0 delet
25.0 delet
35.0 delet
38.0 delet
31.0 delet
40.0 delet
mask (1024, 1024) uint8
(28, 28, 28) <class 'tuple'>
339 (520, 696, 3) uint8
mask (520, 696) uint8
(29, 29, 29) <class 'tuple'>
340 (520, 696, 3) uint8
29.0 

26.0 delet
mask (520, 696) uint8
(8, 8, 8) <class 'tuple'>
417 (520, 696, 3) uint8
40.0 delet
mask (520, 696) uint8
(9, 9, 9) <class 'tuple'>
418 (360, 360, 3) uint8
21.0 delet
mask (360, 360) uint8
(207, 157, 177) <class 'tuple'>
419 (256, 320, 3) uint8
mask (256, 320) uint8
(11, 11, 11) <class 'tuple'>
420 (256, 256, 3) uint8
32.0 delet
mask (256, 256) uint8
(24, 24, 24) <class 'tuple'>
421 (520, 696, 3) uint8
mask (520, 696) uint8
(16, 16, 16) <class 'tuple'>
422 (256, 256, 3) uint8
28.0 delet
mask (256, 256) uint8
(17, 17, 17) <class 'tuple'>
423 (256, 256, 3) uint8
21.0 delet
21.0 delet
21.0 delet
21.0 delet
mask (256, 256) uint8
(14, 14, 14) <class 'tuple'>
424 (360, 360, 3) uint8
mask (360, 360) uint8
(11, 11, 11) <class 'tuple'>
425 (256, 256, 3) uint8
21.0 delet
26.0 delet
mask (256, 256) uint8
(11, 11, 11) <class 'tuple'>
426 (360, 360, 3) uint8
mask (360, 360) uint8
(34, 34, 34) <class 'tuple'>
427 (256, 320, 3) uint8
mask (256, 320) uint8
(3, 3, 3) <class 'tuple'>
428 (256,

mask (360, 360) uint8
(4, 4, 4) <class 'tuple'>
501 (603, 1272, 3) uint8
37.0 delet
40.0 delet
37.0 delet
26.0 delet
33.0 delet
28.0 delet
33.0 delet
36.0 delet
37.0 delet
40.0 delet
22.0 delet
mask (603, 1272) uint8
(13, 13, 13) <class 'tuple'>
502 (520, 696, 3) uint8
mask (520, 696) uint8
(22, 22, 22) <class 'tuple'>
503 (520, 696, 3) uint8
mask (520, 696) uint8
(5, 5, 5) <class 'tuple'>
504 (256, 256, 3) uint8
21.0 delet
24.0 delet
26.0 delet
21.0 delet
29.0 delet
mask (256, 256) uint8
(7, 7, 7) <class 'tuple'>
505 (256, 256, 3) uint8
24.0 delet
26.0 delet
24.0 delet
mask (256, 256) uint8
(12, 12, 12) <class 'tuple'>
506 (256, 256, 3) uint8
29.0 delet
mask (256, 256) uint8
(13, 13, 13) <class 'tuple'>
507 (256, 256, 3) uint8
mask (256, 256) uint8
(17, 17, 17) <class 'tuple'>
508 (256, 256, 3) uint8
21.0 delet
28.0 delet
21.0 delet
21.0 delet
21.0 delet
39.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
25.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
21.0 delet
30.0 delet


36.0 delet
mask (520, 696) uint8
(20, 20, 20) <class 'tuple'>
588 (256, 256, 3) uint8
mask (256, 256) uint8
(7, 7, 7) <class 'tuple'>
589 (256, 256, 3) uint8
31.0 delet
mask (256, 256) uint8
(198, 140, 164) <class 'tuple'>
590 (256, 320, 3) uint8
mask (256, 320) uint8
(5, 5, 5) <class 'tuple'>
591 (512, 640, 3) uint8
mask (512, 640) uint8
(10, 10, 10) <class 'tuple'>
592 (360, 360, 3) uint8
30.0 delet
32.0 delet
39.0 delet
mask (360, 360) uint8
(11, 11, 11) <class 'tuple'>
593 (256, 256, 3) uint8
mask (256, 256) uint8
(197, 123, 153) <class 'tuple'>
594 (256, 320, 3) uint8
33.0 delet
mask (256, 320) uint8
(3, 3, 3) <class 'tuple'>
595 (256, 256, 3) uint8
mask (256, 256) uint8
(9, 9, 9) <class 'tuple'>
596 (520, 696, 3) uint8
mask (520, 696) uint8
(22, 22, 22) <class 'tuple'>
597 (256, 256, 3) uint8
32.0 delet
mask (256, 256) uint8
(193, 154, 177) <class 'tuple'>
598 (256, 320, 3) uint8
mask (256, 320) uint8
(4, 4, 4) <class 'tuple'>
599 (256, 256, 3) uint8
39.0 delet
mask (256, 256) ui

In [ ]:
#!cp -r root1+TEST_PATH root2+TEST_PATH